In [52]:
# %% [markdown]
# ## Setup & Install (jalankan sekali)

In [53]:
%pip install yfinance pandas numpy scikit-learn pandas_ta requests python-dotenv TA-Lib

Note: you may need to restart the kernel to use updated packages.


In [54]:
# %% [markdown]
# ## Konfigurasi

In [55]:
# %%
import time
import datetime as dt
from typing import List, Dict, Tuple, Set
import pandas as pd
import numpy as np
import yfinance as yf
import requests
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
import talib as ta
import warnings
warnings.filterwarnings("ignore")

# --- CONFIG ---
# Link Google Sheets (publik atau di-share sebagai csv). Pastikan sheet 'stocks' dipublikasi sebagai csv.
SHEETS_CSV_URL = "https://docs.google.com/spreadsheets/d/1ePm8eJZK2LX4ep_NMfettEkJzVXt5umfPZjtAv458To/export?format=csv&sheet=stocks"

# Telegram (user provided token). Ganti CHAT_ID dengan id chat tujuan.
TELEGRAM_TOKEN = "8490942127:AAFmqVKPaCpMLBV1rao-mpBFp8NmUESX2QU"
TELEGRAM_CHAT_ID = "-1001942029882"  # contoh: 123456789  (cara dapat: kirim pesan ke bot lalu panggil https://api.telegram.org/bot<TOKEN>/getUpdates)

# Loop timing: main loop tiap 60 detik, summary interval 10 menit
LOOP_SLEEP_SECONDS = 60
SUMMARY_INTERVAL_SECONDS = 10 * 60

# Indicator / scoring thresholds (default — tweak untuk sensitifitas)
SCORE_THRESHOLD = 70   # >= => masuk ARA
VOLUME_SPIKE_MULTIPLIER = 3.0
BREAKOUT_WINDOW = 20
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70

# ML settings (optional). Jika tidak ingin ML, set USE_ML = False
USE_ML = False
ML_LOOKAHEAD_DAYS = 1          # label: return next 1 day > threshold
ML_POS_RETURN_THRESHOLD = 0.02 # 2% next-day gain considered "positive"


In [56]:
# %% [markdown]
# ## Utilities: fetch tickers from Google Sheets, fetch OHLCV


In [57]:
# %%
def fetch_tickers_from_sheet(csv_url: str) -> List[str]:
    """
    Google Sheet tanpa header — ambil semua ticker dari kolom pertama.
    """
    # header=None agar kolom terbaca sebagai 0,1,2...
    df = pd.read_csv(csv_url, header=None)

    # kolom pertama = daftar ticker
    tickers = (
        df.iloc[:, 0]
        .dropna()
        .astype(str)
        .str.strip()
        .str.replace(" ", "")
        .tolist()
    )

    # filter entries yang valid, misal tidak kosong & panjang minimal
    tickers = [t for t in tickers if t not in ["", " ", None]]

    return tickers



def fetch_history(ticker, period="6mo", interval="1d"):
    try:
        data = yf.download(
            ticker,
            period=period,
            interval=interval,
            progress=False,
            auto_adjust=False,
            group_by="column"   # penting: cegah multi-index
        )

        if data.empty:
            return pd.DataFrame()

        # If multi-index columns → flatten
        if isinstance(data.columns, pd.MultiIndex):
            data.columns = [c[0].lower() for c in data.columns]
        else:
            # normal case
            data.columns = [c.lower() for c in data.columns]

        return data

    except Exception as e:
        print("Fetch error for", ticker, e)
        return pd.DataFrame()



In [58]:
# %% [markdown]
# ## Indikator teknikal & bandarmology


In [59]:
# %%
def compute_indicators(df):
    try:
        if df is None or df.empty or len(df) < 30:
            return pd.DataFrame()

        df = df.copy()

        close = df['close'].astype(float)
        high  = df['high'].astype(float)
        low   = df['low'].astype(float)
        vol   = df['volume'].astype(float) if 'volume' in df else pd.Series(0, index=df.index)

        # TA-Lib indicators (wrapped to avoid error)
        try:
            df['sma_10'] = ta.SMA(close, timeperiod=10)
            df['sma_20'] = ta.SMA(close, timeperiod=20)
            df['ema_12'] = ta.EMA(close, timeperiod=12)
            df['ema_26'] = ta.EMA(close, timeperiod=26)

            macd, macd_signal, _ = ta.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
            df['macd'] = macd
            df['macd_signal'] = macd_signal

            df['rsi_14'] = ta.RSI(close, timeperiod=14)

            upper, middle, lower = ta.BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2)
            df['bb_upper']  = upper
            df['bb_middle'] = middle
            df['bb_lower']  = lower

        except Exception:
            # TA-Lib failed for this ticker → return empty df
            return pd.DataFrame()

        # Volume-based indicators
        df['vol_ma_20'] = vol.rolling(20).mean()
        df['vol_spike'] = vol / (df['vol_ma_20'] + 1e-9)

        # Breakout
        df['rolling_high_20'] = high.rolling(20).max()
        df['is_breakout'] = df['close'] > df['rolling_high_20']

        # Change %
        df['pct_change'] = df['close'].pct_change() * 100

        # Cleanup
        df.ffill(inplace=True)
        df.fillna(0, inplace=True)

        return df

    except Exception:
        return pd.DataFrame()





def compute_signal(df):
    if df is None or df.empty or len(df) < 30:
        return {"score": 0, "reasons": [], "is_ara": False}

    try:
        last = df.iloc[-1]
    except:
        return {"score": 0, "reasons": [], "is_ara": False}

    score = 0
    reasons = []

    try:
        if last['macd'] > last['macd_signal']:
            score += 15; reasons.append("MACD Bullish")
    except:
        pass  # MACD missing

    try:
        if last['close'] > last['sma_20']:
            score += 10; reasons.append("Above SMA20")
    except:
        pass

    try:
        if last['ema_12'] > last['ema_26']:
            score += 10; reasons.append("EMA Bullish")
    except:
        pass

    try:
        if last['rsi_14'] < RSI_OVERBOUGHT:
            score += 5
        if last['rsi_14'] < RSI_OVERSOLD:
            score += 5; reasons.append("RSI Oversold")
    except:
        pass

    try:
        if last['vol_spike'] > VOLUME_SPIKE_MULTIPLIER:
            score += 20; reasons.append("Volume Spike")
    except:
        pass

    try:
        if last['is_breakout']:
            score += 20; reasons.append("Breakout")
    except:
        pass

    try:
        if last['close'] > last['bb_upper']:
            score += 10; reasons.append("BB Upper Break")
    except:
        pass

    score = max(0, min(100, score))
    return {"score": score, "reasons": reasons, "is_ara": score >= SCORE_THRESHOLD}



def compute_ATR(df):
    return ta.ATR(df['high'], df['low'], df['close'], timeperiod=14)


In [60]:
# %%

def bandarmology_predictive_model(df):
    try:
        if df is None or df.empty or len(df) < 25:
            return {"prob": 0, "reason": ["insufficient data"]}

        df = df.copy()
        last = df.iloc[-1]
        prev = df.iloc[-5:]

        score = 0
        reasons = []

        # Accumulation pressure
        try:
            candle_body = abs(last['close'] - df['open'].iloc[-1])
            ap = last['volume'] / (candle_body + 1e-9)
            if ap > prev['volume'].mean() * 1.2:
                score += 15; reasons.append("Accumulation Pressure")
        except:
            pass

        # Volume anomaly
        try:
            vsn = last['vol_spike']
            if vsn > 2:
                score += min(20, vsn * 3)
                reasons.append(f"Volume Spike x{vsn:.1f}")
        except:
            pass

        # Breakout zone
        try:
            level = df['rolling_high_20'].iloc[-2]
            prox = last['close'] / (level + 1e-9)
            if 0.97 < prox < 1.02:
                score += 20; reasons.append("Near Breakout Zone")
        except:
            pass

        # Squeeze (ATR)
        try:
            atr = ta.ATR(df['high'], df['low'], df['close'], timeperiod=14)
            atr_ratio = atr.iloc[-1] / atr.iloc[-10:].mean()
            if atr_ratio < 0.8:
                score += 10; reasons.append("Volatility Squeeze")
        except:
            pass

        prob = max(0, min(100, score))
        return {"prob": prob, "reason": reasons}

    except Exception:
        return {"ticker": t ,"prob": 0, "reason": ["model error"]}



def build_dataset_for_ml(tickers: List[str], lookback="1y", interval="1d") -> pd.DataFrame:
    """
    Ambil data untuk semua tickers, hitung indikator, dan gabungkan jadi dataset fitur.
    Label = 1 jika return next ML_LOOKAHEAD_DAYS >= ML_POS_RETURN_THRESHOLD
    """
    rows = []
    for t in tickers:
        df = fetch_history(t, period=lookback, interval=interval)
        if df.empty or len(df) < 60:
            continue
        df = compute_indicators(df)
        df = df.reset_index().rename(columns={'index':'date'})
        # create features from each date
        for i in range(len(df)-ML_LOOKAHEAD_DAYS):
            cur = df.loc[i]
            future = df.loc[i+ML_LOOKAHEAD_DAYS]
            # label
            ret = (future['close'] - cur['close']) / (cur['close'] + 1e-9)
            label = 1 if ret >= ML_POS_RETURN_THRESHOLD else 0
            feat = {
                "ticker": t,
                "date": cur['date'],
                "close": cur['close'],
                "sma_10": cur['sma_10'],
                "sma_20": cur['sma_20'],
                "ema_12": cur['ema_12'],
                "ema_26": cur['ema_26'],
                "macd": cur['macd'],
                "macd_signal": cur['macd_signal'],
                "rsi_14": cur['rsi_14'],
                "vol_spike": cur['vol_spike'],
                "is_breakout": int(cur['is_breakout']),
                "pct_change": cur['pct_change'],
                "label": label
            }
            rows.append(feat)
    if not rows:
        return pd.DataFrame()
    return pd.DataFrame(rows)

def train_ml_model(df_features: pd.DataFrame):
    """
    Latih RandomForest classifier, kembalikan model dan scaler.
    """
    if df_features.empty:
        return None, None
    X = df_features[['close','sma_10','sma_20','ema_12','ema_26','macd','macd_signal','rsi_14','vol_spike','is_breakout','pct_change']].fillna(0)
    y = df_features['label']
    scaler = StandardScaler()
    Xs = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.2, random_state=42, stratify=y)
    model = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1, class_weight='balanced')
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    # cross val
    cv = cross_val_score(model, Xs, y, cv=3, scoring='accuracy')
    print(f"ML model trained. Test acc: {score:.3f}. CV mean acc: {cv.mean():.3f}")
    return model, scaler


In [61]:
# %% [markdown]
# ## Telegram notification helper


In [62]:
# %%
def send_telegram_message(token: str, chat_id: str, text: str) -> bool:
    url = f"https://api.telegram.org/bot{token}/sendMessage"
    payload = {"chat_id": chat_id, "text": text, "parse_mode": "HTML"}
    try:
        r = requests.post(url, data=payload, timeout=10)
        return r.status_code == 200
    except Exception as e:
        print(f"Telegram send error: {e}")
        return False


In [63]:
# %% [markdown]
# ## Monitor loop: check setiap menit, kirim notif sesuai aturan


In [64]:
def send_telegram(text):
    url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"
    r = requests.post(url, data={"chat_id": TELEGRAM_CHAT_ID, "text": text, "parse_mode":"HTML"})
    return r.status_code == 200


def format_telegram_message(ara_today, ara_tomorrow, stats):
    lines = []

    # ARA TODAY
    if ara_today:
        lines.append("🚨 <b>ARA TODAY DETECTED</b>")
        for t, s in ara_today.items():
            lines.append(f"- {t} (score: {s['score']})")
            lines.append(f"  Reasons: {', '.join(s['reasons'])}")
    else:
        lines.append("✓ Tidak ada ARA hari ini.")

    lines.append("")  # spacer

    # ARA TOMORROW PREDICTION
    lines.append("📈 <b>ARA PROBABILITY FOR TOMORROW</b>")

    if ara_tomorrow:
        rank = 1
        for t, p in ara_tomorrow:
            lines.append(f"{rank}. {t} → {p['prob']}%")
            lines.append(f"   Reasons: {', '.join(p['reason'])}")
            rank += 1
    else:
        lines.append("Tidak ada kandidat probabilitas tinggi.")

    lines.append("")

    # Stats
    lines.append("📊 <b>STATUS</b>")
    lines.append(f"- Total scanned: {stats['total']}")
    lines.append(f"- ARA today: {stats['ara_today']}")
    lines.append(f"- High-probability tomorrow (>50%): {stats['high_prob']}")

    return "\n".join(lines)


In [65]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def process_ticker(t):
    try:
        df = fetch_history(t)
        dfb = fetch_history(t, period="3mo")
        df = compute_indicators(df)
        if df.empty:
            return t, None, None

        sig_today = compute_signal(df)
        pred_tomorrow = bandarmology_predictive_model(dfb)
        print(t, sig_today, pred_tomorrow)

        return t, sig_today, pred_tomorrow

    except:
        return t, None, None


def parallel_scan(tickers, max_workers=20):
    results_today = {}
    predictions = {}

    with ThreadPoolExecutor(max_workers=max_workers) as exe:
        futures = {exe.submit(process_ticker, t): t for t in tickers}

        for fut in as_completed(futures):
            t = futures[fut]
            try:
                _, today, pred = fut.result()

                if today:
                    results_today[t] = today
                if pred:
                    predictions[t] = pred

            except:
                continue

    return results_today, predictions


In [66]:
# %%
from datetime import datetime 


def predict_ara_tomorrow(tickers):
    result = {}

    for t in tickers:
        df = fetch_history(t, period="3mo")
        df = compute_indicators(df)

        pred = bandarmology_predictive_model(df)
        result[t] = pred

        print(pred)

    return result

def predict_tomorrow_for_all(tickers):
    predictions = {}

    for t in tickers:
        try:
            df = fetch_history(t, period="3mo")
            df = compute_indicators(df)
            pred = bandarmology_predictive_model(df)
            predictions[t] = pred
            print(pred)
        except:
            continue  # SKIP ticker error

    sorted_preds = sorted(predictions.items(), key=lambda x: x[1]['prob'], reverse=True)
    return sorted_preds




# Main loop runner (run this cell to start monitoring)
def run_monitor_loop():
    tickers = fetch_tickers_from_sheet(SHEETS_CSV_URL)
    print(f"Loaded {len(tickers)} tickers.")

    last_sent_summary_at = dt.datetime.now(dt.UTC)
    last_ara_set = set()

    while True:
        now = dt.datetime.now(dt.UTC)

        ara_today = {}
        results_today = {}
        predictions = {}

        # --- SINGLE PASS SCAN ---
        for t in tickers:
            try:
                # Fetch only once
                df = fetch_history(t, period="6mo")
                if df.empty:
                    continue

                # Compute indicators only once
                df = compute_indicators(df)
                if df.empty:
                    continue

                # 1️⃣ ARA today
                sig = compute_signal(df)
                results_today[t] = sig
                print(t, sig)

                if sig["is_ara"]:
                    ara_today[t] = sig

                # 2️⃣ Predict ARA tomorrow using the SAME df
                pred = bandarmology_predictive_model(df)
                predictions[t] = pred
                print(t, pred)

            except:
                continue

        # --- Now compute sorted predictions ---
        sorted_preds = sorted(predictions.items(), key=lambda x: x[1]['prob'], reverse=True)
        top_predictions = [(t,p) for t,p in sorted_preds[:10] if p['prob'] >= 20]
        high_prob_count = len([p for _,p in sorted_preds if p['prob'] >= 50])

        stats = {
            "total": len(tickers),
            "ara_today": len(ara_today),
            "high_prob": high_prob_count
        }

        # --- Decide if notification should be sent ---
        ara_now = set(ara_today.keys())
        new_ara = ara_now - last_ara_set

        send = False

        if new_ara:
            send = True
        elif (now - last_sent_summary_at).total_seconds() >= SUMMARY_INTERVAL_SECONDS:
            send = True

        if send:
            msg = format_telegram_message(ara_today, top_predictions, stats)
            send_telegram(msg)
            last_sent_summary_at = now

        last_ara_set = ara_now
        time.sleep(LOOP_SLEEP_SECONDS)


def run_monitor_loop_parallel():
    tickers = fetch_tickers_from_sheet(SHEETS_CSV_URL)
    print(f"Loaded {len(tickers)} tickers.")

    last_ara_set = set()
    last_sent = dt.datetime.now(dt.UTC)

    while True:
        now = dt.datetime.now(dt.UTC)

        # 🚀 super-fast single-pass
        results_today, predictions = parallel_scan(tickers)

        ara_today = {t:s for t,s in results_today.items() if s["is_ara"]}
        ara_now = set(ara_today.keys())
        new_ara = ara_now - last_ara_set

        sorted_preds = sorted(predictions.items(), key=lambda x:x[1]["prob"], reverse=True)
        high_prob_count = len([x for x in sorted_preds if x[1]["prob"] >= 50])

        stats = {
            "total": len(tickers),
            "ara_today": len(ara_today),
            "high_prob": high_prob_count
        }

        print(stats)

        send = False
        if new_ara:
            send = True
        elif (now - last_sent).total_seconds() >= SUMMARY_INTERVAL_SECONDS:
            send = True

        if send:
            msg = format_telegram_message(ara_today, sorted_preds, stats)
            send_telegram(msg)
            last_sent = now

        last_ara_set = ara_now
        time.sleep(LOOP_SLEEP_SECONDS)


In [67]:
# %% [markdown]
# ## Cara pakai / contoh


In [68]:
# %%
# 1) Pastikan SHEETS_CSV_URL benar dan file memiliki kolom 'ticker' atau 'symbol'.
# 2) Isi TELEGRAM_CHAT_ID dengan chat_id kamu.
# 3) Jalankan run_monitor_loop() di cell ini untuk memulai loop.


# tickers = fetch_tickers_from_sheet(SHEETS_CSV_URL)
# predictions = predict_ara_tomorrow(tickers[:50])   # test dulu 50 ticker

# sorted_preds = sorted(predictions.items(), key=lambda x: x[1]['prob'], reverse=True)

# for t, p in sorted_preds[:10]:
#     print(f"{t}: {p['prob']}% chance ARA tomorrow → {p['reason']}")

# EXAMPLE_TICKER = "PJHB.JK"  # contoh, ganti sesuai daftar kamu
# df_test = fetch_history(EXAMPLE_TICKER, period="6mo", interval="1d")
# print("Raw columns:", df_test.columns.tolist())
# df_ind = compute_indicators(df_test)

# signal = compute_signal(df_ind)

# print("Computed columns (indicator):", [c for c in df_ind.columns if c.startswith(('sma','ema','macd','rsi','bb','vol'))])

# print(signal)
# print("Computed columns (sample):", [c for c in df_ind.columns if c.startswith(('sma','ema','macd','rsi','bb','vol'))])
# print(df_ind[['close','vol_ma_20','vol_spike','is_breakout']].tail(5))

run_monitor_loop()
# run_monitor_loop_parallel()


Loaded 784 tickers.
AALI.JK {'score': 5, 'reasons': [], 'is_ara': False}
AALI.JK {'prob': 0, 'reason': []}
ABMM.JK {'score': 5, 'reasons': [], 'is_ara': False}
ABMM.JK {'prob': 0, 'reason': []}
ACES.JK {'score': 5, 'reasons': [], 'is_ara': False}
ACES.JK {'prob': 0, 'reason': []}
ACST.JK {'score': 15, 'reasons': ['EMA Bullish'], 'is_ara': False}
ACST.JK {'prob': 0, 'reason': []}
ADES.JK {'score': 15, 'reasons': ['EMA Bullish'], 'is_ara': False}
ADES.JK {'prob': 0, 'reason': []}
ADHI.JK {'score': 5, 'reasons': [], 'is_ara': False}
ADHI.JK {'prob': 0, 'reason': []}
ADMF.JK {'score': 10, 'reasons': ['RSI Oversold'], 'is_ara': False}
ADMF.JK {'prob': 0, 'reason': []}
ADMG.JK {'score': 50, 'reasons': ['MACD Bullish', 'Above SMA20', 'EMA Bullish', 'BB Upper Break'], 'is_ara': False}
ADMG.JK {'prob': 20, 'reason': ['Near Breakout Zone']}
ADRO.JK {'score': 5, 'reasons': [], 'is_ara': False}
ADRO.JK {'prob': 15, 'reason': ['Accumulation Pressure']}
AGII.JK {'score': 40, 'reasons': ['MACD Bullis

KeyboardInterrupt: 